## 处理A股数据

In [ ]:
import pandas as pd
import pickle
from tqdm import tqdm


In [ ]:

new_data = pickle.load(open(r'/home/xuyang1/PatchTST/our_align_data.lm.V06.pickle/new_data.pkl', 'rb'))

# data = pickle.load(open(r'/home/xuyang1/PatchTST/our_align_data.lm.V06.pickle/share.pkl', 'rb'))
# ret = pickle.load(open(r'/home/xuyang1/PatchTST/ret.pkl', 'rb'))

# ret = ret.reindex(data.index)
# new_data = pd.concat([data.iloc(axis=1)[:-5], ret], axis=1)

In [ ]:
# for col in tqdm(new_data.columns):
#     tmp_df = pd.DataFrame({"left": new_data[col], "right": new_data.iloc(axis=1)[-5]})
#     ic1 = tmp_df.groupby("datetime").apply(lambda df: df["left"].corr(df["right"])).mean()
#     tmp_df = pd.DataFrame({"left": new_data[col], "right": new_data.iloc(axis=1)[-3]})
#     ic2 = tmp_df.groupby("datetime").apply(lambda df: df["left"].corr(df["right"])).mean()
#     if ic1 < 0 and ic2 < 0:
#         new_data[col] *= -1
#         print(col, ic1.mean(), ic2.mean())
    # assert ic1 > 0 == ic2 > 0

#     ic = df.groupby("datetime").apply(lambda df: df["pred"].corr(df["label"]))


# for index, col in enumerate(data.columns):
#     print(index, col)
# for index, col in enumerate(ret.columns):
#     print(index, col)

# for index in range(5):
#     pred = data.iloc[:, -5 + index]
#     label = ret.iloc[:, index]
#     label = label.reindex(pred.index)
#     print(pred.shape, label.shape)
#     df = pd.DataFrame({"pred": pred, "label": label})
#     ic = df.groupby("datetime").apply(lambda df: df["pred"].corr(df["label"]))
#     ric = df.groupby("datetime").apply(lambda df: df["pred"].corr(df["label"], method="spearman"))
#     print(ic.mean(), ric.mean())

In [ ]:

display(new_data.shape)
display(new_data.head())
# pickle.dump(new_data, open(r'/home/xuyang1/PatchTST/our_align_data.lm.V06.pickle/new_data.pkl', 'wb'))

In [ ]:
from pandarallel import pandarallel
pandarallel.initialize()

tqdm.pandas()
def calculate_quintile_ret(df):
    label = df.iloc(axis=1)[-5]
    quintile_ret = pd.Series(index=df.columns)
    for col in df.columns:
        tmp_df = pd.DataFrame({"left": df[col], "right": label}).sort_values("left", ascending=False)
        group1_ret = tmp_df.iloc[:int(len(tmp_df) * 0.2), 1].mean()
        group5_ret = tmp_df.iloc[int(len(tmp_df) * 0.8):, 1].mean()
        quintile_ret[col] = group1_ret - group5_ret

    return quintile_ret

# quintile_ret_data = new_data.groupby("datetime").parallel_apply(calculate_quintile_ret)
# pickle.dump(quintile_ret_data, open(r'/home/xuyang1/PatchTST/our_align_data.lm.V06.pickle/quintile_ret_data.pkl', 'wb'))

In [ ]:
from pandarallel import pandarallel
pandarallel.initialize()

def calculate_top10p_ret(df):
    label = df.iloc(axis=1)[-5]
    top10p_ret = pd.Series(index=df.columns)
    for col in df.columns:
        tmp_df = pd.DataFrame({"left": df[col], "right": label}).sort_values("left", ascending=False)
        top10p_ret[col] = tmp_df.iloc[:int(len(tmp_df) * 0.1), 1].mean()
    return top10p_ret

# top10p_ret_data = new_data.groupby("datetime").parallel_apply(calculate_top10p_ret)
# pickle.dump(top10p_ret_data, open(r'/home/xuyang1/PatchTST/our_align_data.lm.V06.pickle/top10p_ret_data.pkl', 'wb'))

In [ ]:

# top10p_ret_data = pickle.load(open(r'/home/xuyang1/PatchTST/our_align_data.lm.V06.pickle/top10p_ret_data.pkl', 'rb'))
# top10p_ret_data.columns = [col[1] for col in top10p_ret_data.columns]
# import plotly.express as px
# top10p_ret_data_cumsum = top10p_ret_data.cumsum().iloc(axis=1)[::5]
# fig = px.line(top10p_ret_data_cumsum, title="Top 10% cumulative return")
# fig.show()


In [ ]:
quintile_ret_data = pickle.load(open(r'/home/xuyang1/PatchTST/our_align_data.lm.V06.pickle/quintile_ret_data.pkl', 'rb'))
top10p_ret_data = pickle.load(open(r'/home/xuyang1/PatchTST/our_align_data.lm.V06.pickle/top10p_ret_data.pkl', 'rb'))

quintile_ret_data_nolabel = quintile_ret_data.iloc(axis=1)[:-5]
top10p_ret_data_nolabel = top10p_ret_data.iloc(axis=1)[:-5]
quintile_ret_data_nolabel.columns = [col[1] for col in quintile_ret_data_nolabel.columns]
top10p_ret_data_nolabel.columns = [col[1] for col in top10p_ret_data_nolabel.columns]
quintile_ret_data_nolabel.index.names = ["date"]
top10p_ret_data_nolabel.index.names = ["date"]

quintile_ret_data_nolabel.to_csv(r'/home/xuyang1/PatchTST/PatchTST_supervised/dataset/quintile_stocka.csv')
top10p_ret_data_nolabel.to_csv(r'/home/xuyang1/PatchTST/PatchTST_supervised/dataset/top10p_us.csv')


In [ ]:
import os
shape_series = pd.Series()
import plotly.express as px
for name in os.listdir(r'/home/xuyang1/PatchTST/PatchTST_supervised/dataset/'):
    tmp = pd.read_csv(os.path.join(r'/home/xuyang1/PatchTST/PatchTST_supervised/dataset/', name))
    shape_series[name] = tmp.shape
    if "stocka" in name:
        tmp = tmp.set_index("date")
        tmp = tmp.rolling(20).mean()
        fig = px.line(tmp.iloc(axis=1)[::10], title=f"{name} Top 10% cumulative return")
        fig.show()
    
# display(shape_series.to_frame("dataset_size"))

## 验证结果

In [ ]:
import pandas as pd
import os
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import pickle
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

label_stocka_quintile = pd.read_csv(r'/home/xuyang1/PatchTST/PatchTST_supervised/dataset/quintile_stocka.csv', index_col=0)
label_us_quintile = pd.read_csv(r'/home/xuyang1/PatchTST/PatchTST_supervised/dataset/quintile_us.csv', index_col=0)
# label_stocka_quintile = pd.DataFrame(pickle.load(open(r'/home/xuyang1/PatchTST/PatchTST_supervised/dataset/quintile_stocka_transformed.pkl', 'rb')), index=label_stocka_quintile.index, columns=label_stocka_quintile.columns)
# label_us_quintile = pd.DataFrame(pickle.load(open(r'/home/xuyang1/PatchTST/PatchTST_supervised/dataset/quintile_us_transformed.pkl', 'rb')), index=label_us_quintile.index, columns=label_us_quintile.columns)
label_stocka_top10p = pd.read_csv(r'/home/xuyang1/PatchTST/PatchTST_supervised/dataset/top10p_stocka.csv', index_col=0)
label_us_top10p = pd.read_csv(r'/home/xuyang1/PatchTST/PatchTST_supervised/dataset/top10p_us.csv', index_col=0)
# label_stocka_top10p = pd.DataFrame(pickle.load(open(r'/home/xuyang1/PatchTST/PatchTST_supervised/dataset/top10p_stocka_transformed.pkl', 'rb')), index=label_stocka_top10p.index, columns=label_stocka_top10p.columns)
# label_us_top10p = pd.DataFrame(pickle.load(open(r'/home/xuyang1/PatchTST/PatchTST_supervised/dataset/top10p_us_transformed.pkl', 'rb')), index=label_us_top10p.index, columns=label_us_top10p.columns)

quintile_stocka_scaler = pickle.load(open(r'/home/xuyang1/PatchTST/PatchTST_supervised/dataset/quintile_stocka_scaler.pkl', 'rb'))
top10p_stocka_scaler = pickle.load(open(r'/home/xuyang1/PatchTST/PatchTST_supervised/dataset/top10p_stocka_scaler.pkl', 'rb'))
quintile_us_scaler = pickle.load(open(r'/home/xuyang1/PatchTST/PatchTST_supervised/dataset/quintile_us_scaler.pkl', 'rb'))
top10p_us_scaler = pickle.load(open(r'/home/xuyang1/PatchTST/PatchTST_supervised/dataset/top10p_us_scaler.pkl', 'rb'))
losses = []
for name in os.listdir(r"/home/xuyang1/PatchTST/PatchTST_supervised/results"):
    model = name.split("_")[4]
    seq_length = name.split("_")[3]
    dataset = name.split("_")[0]
    dataset_country = name.split("_")[1]
    if dataset == "quintile" and dataset_country == "stocka":
        label = label_stocka_quintile
    elif dataset == "quintile" and dataset_country == "us":
        label = label_us_quintile
    elif dataset == "top10p" and dataset_country == "stocka":
        label = label_stocka_top10p
    elif dataset == "top10p" and dataset_country == "us":
        label = label_us_top10p
    pred = np.load(rf"/home/xuyang1/PatchTST/PatchTST_supervised/results/{name}/pred.npy")

    if dataset == "quintile" and dataset_country == "stocka":
        scaler = quintile_stocka_scaler
    elif dataset == "quintile" and dataset_country == "us":
        scaler = quintile_us_scaler
    elif dataset == "top10p" and dataset_country == "stocka":
        scaler = top10p_stocka_scaler
    elif dataset == "top10p" and dataset_country == "us":
        scaler = top10p_us_scaler
    pred = scaler.inverse_transform(np.reshape(pred, (-1, pred.shape[-1]))).reshape(pred.shape)

    # if "quinti" not in dataset or seq_length != "96":
    #     continue

    test_length = int(len(label) * 0.2)

    for i in range(pred.shape[-1]):
        if label.values[:-test_length,i].mean() < 0:
            label.iloc[:,i] = label.iloc[:,i] * -1
            pred[:,:,i] = -pred[:,:,i]
    target_label = label.iloc[-test_length:,:]
    if pred.shape[0] > target_label.shape[0]:
        pred = pred[:target_label.shape[0],:,:]
    elif pred.shape[0] < target_label.shape[0]:
        target_label = target_label[:pred.shape[0],:]


    # calculate pred to real label loss:
    # loss = 0
    # for i in range(pred.shape[0]):
    #     pred_batch = pred[i]
    #     label_batch = target_label.iloc[i:i+pred.shape[1],:].values
    #     loss += np.mean((pred_batch - label_batch)**2)

    # calculate real return with prediction:
    # 1. use closest pred as real pread
    # real_pred = pred[:,0,:]

    # 2. use mean of pred as real pred
    # pred_middata_np = np.zeros((pred.shape[0], pred.shape[0] + pred.shape[1] - 1, pred.shape[2]))
    # pred_middata_np[:,:,:] = np.nan
    # for i in range(pred.shape[0]):
    #     pred_middata_np[i, i:i+pred.shape[1], :] = pred[i]
    # real_pred = np.nanmean(pred_middata_np, axis=0, keepdims=False)

    # 3. use mean of pred as real pred, but only use the first 20 pred
    real_pred = pred[:, :20, :].mean(axis=1, keepdims=False)

    pred_df = pd.DataFrame(real_pred, index=target_label.index[:real_pred.shape[0]], columns=target_label.columns)

    # ---- calculate rolling 20 days return ----
    rolling_20_label = target_label.rolling(20).mean().shift(-19).iloc(axis=0)[:-19]
    average_ret = rolling_20_label.mean(axis=1).to_frame("average_ret")

    past_rolling_20_label = target_label.rolling(20, closed="left", min_periods=1).mean()

    follow_past_20days_50p = rolling_20_label[(past_rolling_20_label.rank(axis=1, pct=True) > 0.5)].mean(axis=1).to_frame("follow_past_20days_50p")
    follow_past_20days_20p = rolling_20_label[(past_rolling_20_label.rank(axis=1, pct=True) > 0.8)].mean(axis=1).to_frame("follow_past_20days_20p")
    follow_past_20days_10p = rolling_20_label[(past_rolling_20_label.rank(axis=1, pct=True) > 0.9)].mean(axis=1).to_frame("follow_past_20days_10p")

    rolling_20_pred_df = pred_df.rolling(20).mean().shift(-20).iloc(axis=0)[:-20]
    top50p_pred_ret = rolling_20_label[(pred_df.rank(axis=1, pct=True) > 0.5)].mean(axis=1).to_frame("top50p_pred_ret")
    top20p_pred_ret = rolling_20_label[(pred_df.rank(axis=1, pct=True) > 0.8)].mean(axis=1).to_frame("top20p_pred_ret")
    top10p_pred_ret = rolling_20_label[(pred_df.rank(axis=1, pct=True) > 0.9)].mean(axis=1).to_frame("top10p_pred_ret")
    show_df = pd.concat([average_ret, top50p_pred_ret, top20p_pred_ret, top10p_pred_ret, follow_past_20days_50p, follow_past_20days_20p, follow_past_20days_10p], axis=1).cumsum(axis=0)
    show_df.plot()
    plt.title(f"{dataset_country} {dataset} {model} {seq_length}")
    # plt.show()

    # ---- plot pred and label in same photo ----
    # pred_with_input = pd.concat([label.iloc[:-test_length,:], pred_df], axis=0)
    # pred_with_input.columns = [col + "_pred" for col in pred_with_input.columns]
    # fig, axes = plt.subplots(5, 5)
    # fig.set_figheight(30)
    # fig.set_figwidth(30)
    # plt.subplots_adjust(wspace=None, hspace=None)
    # axes = axes.reshape(5, 5)
    # stride = label.shape[-1] // 25
    # for i in range(25):
    #     pred_and_gt = pd.concat([label.iloc(axis=1)[i*stride:i*stride+1], pred_with_input.iloc(axis=1)[i*stride:i*stride+1]], axis=1).cumsum(axis=0)
    #     pred_and_gt.plot(ax=axes[i//5, i%5], legend=True)
    #     axes[i//5, i%5].set_title(label.columns[i*stride])
    # plt.title(f"{dataset_country} {dataset} {model} {seq_length}")
    plt.savefig(rf"/home/xuyang1/PatchTST/PatchTST_supervised/plot_result/{dataset_country}_{dataset}_{model}_{seq_length}.png")
    # plt.show()
    
    # rics = []
    # for col in pred_df.columns:
    #     rics.append(pred_df[col].corr(target_label[col], method="spearman"))

    # print(dataset_country, dataset, seq_length, model, sum(rics) / len(rics), sep=",")
    plt.close()

## 处理美股数据


In [ ]:
import pandas as pd
from tqdm import tqdm 
from functools import partial
import pickle


In [ ]:

msn_data = pd.read_parquet(r"/home/xuyang1/PatchTST/es_data/msn_data.parq")
price = pd.read_parquet(r"/home/xuyang1/PatchTST/es_data/orig_price_info.parquet")

msn_data = msn_data.iloc(axis=1)[:-1]
msn_data.columns = [i[1] for i in msn_data.columns]
def guess_price(df, shift, horizon):
    return df.price.shift(-shift-horizon) / df.price.shift(-shift) - 1

tqdm.pandas()
price["price_1d"] = price.groupby("instrument", group_keys=False).progress_apply(partial(guess_price, shift=1, horizon=1)).sort_index()
new_data = msn_data.join(price["price_1d"], how="left").dropna(subset=['price_1d'])

In [ ]:
from pandarallel import pandarallel
pandarallel.initialize()

tqdm.pandas()
def calculate_quintile_ret(df):
    label = df.iloc(axis=1)[-1]
    quintile_ret = pd.Series(index=df.columns)
    for col in df.columns:
        tmp_df = pd.DataFrame({"left": df[col], "right": label}).sort_values("left", ascending=False)
        group1_ret = tmp_df.iloc[:int(len(tmp_df) * 0.2), 1].mean()
        group5_ret = tmp_df.iloc[int(len(tmp_df) * 0.8):, 1].mean()
        quintile_ret[col] = group1_ret - group5_ret

    return quintile_ret

quintile_ret_data = new_data.groupby("datetime").parallel_apply(calculate_quintile_ret)
pickle.dump(quintile_ret_data, open(r'/home/xuyang1/PatchTST/es_data/quintile_ret_data.pkl', 'wb'))

In [ ]:
from pandarallel import pandarallel
pandarallel.initialize()

def calculate_top10p_ret(df):
    label = df.iloc(axis=1)[-1]
    top10p_ret = pd.Series(index=df.columns)
    for col in df.columns:
        tmp_df = pd.DataFrame({"left": df[col], "right": label}).sort_values("left", ascending=False)
        top10p_ret[col] = tmp_df.iloc[:int(len(tmp_df) * 0.1), 1].mean()
    return top10p_ret

top10p_ret_data = new_data.groupby("datetime").parallel_apply(calculate_top10p_ret)
pickle.dump(top10p_ret_data, open(r'/home/xuyang1/PatchTST/es_data/top10p_ret_data.pkl', 'wb'))

In [ ]:
quintile_ret_data = pickle.load(open(r'/home/xuyang1/PatchTST/es_data/quintile_ret_data.pkl', 'rb'))
top10p_ret_data = pickle.load(open(r'/home/xuyang1/PatchTST/es_data/top10p_ret_data.pkl', 'rb'))


In [ ]:
for col in tqdm(quintile_ret_data.columns):
    count = 0
    while True:
        upper = quintile_ret_data[col].mean() + quintile_ret_data[col].std() * 5
        lower = quintile_ret_data[col].mean() - quintile_ret_data[col].std() * 5
        if (upper > quintile_ret_data[col].max()) and (lower < quintile_ret_data[col].min()):
            break
        if count > 3:
            break
        count += 1
        quintile_ret_data[col] = quintile_ret_data[col].clip(lower, upper,)

for col in tqdm(top10p_ret_data.columns):
    count = 0
    while True:
        upper = top10p_ret_data[col].mean() + top10p_ret_data[col].std() * 5
        lower = top10p_ret_data[col].mean() - top10p_ret_data[col].std() * 5
        if (upper > top10p_ret_data[col].max()) and (lower < top10p_ret_data[col].min()):
            break
        if count > 3:
            break
        count += 1
        top10p_ret_data[col] = top10p_ret_data[col].clip(lower, upper,)

In [ ]:
quintile_ret_data = quintile_ret_data.iloc(axis=1)[:-1]
top10p_ret_data = top10p_ret_data.iloc(axis=1)[:-1]

quintile_ret_data.index.names = ["date"]
top10p_ret_data.index.names = ["date"]

quintile_ret_data.to_csv(r'/home/xuyang1/PatchTST/PatchTST_supervised/dataset/quintile_us.csv')
top10p_ret_data.to_csv(r'/home/xuyang1/PatchTST/PatchTST_supervised/dataset/top10p_us.csv')


## 记录所有loss

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
lines = open(r"/home/xuyang1/PatchTST/final_result.txt").readlines()

result_df = pd.DataFrame(columns=["dataset", "country", "seq_length", "model", "mse", "mae", "rse"])
for line in lines:
    line = line.strip()
    if len(line) == 0:
        continue
    dataset = line.split("_")[0]
    dataset_country = line.split("_")[1]
    seq_len = int(line.split("_")[3])
    model = line.split("_")[4]
    mse = float(line.split("mse:")[1].split(",")[0])
    mae = float(line.split("mae:")[1].split(",")[0])
    rse = float(line.split("rse:")[1].split(",")[0])
    result_df = pd.concat([result_df, pd.DataFrame([[dataset, dataset_country, seq_len, model, mse, mae, rse]], columns=result_df.columns)], axis=0)

result_df = result_df.set_index(["dataset", "country", "seq_length", "model"]).sort_index()
result_df.to_excel(r"/home/xuyang1/PatchTST/final_result.xlsx")
# display(result_df)


## 拼接所有图片

In [ ]:
import pandas as pd
import os
import cv2
import numpy as np
from tqdm import tqdm
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
file_names = os.listdir(r"/home/xuyang1/PatchTST/PatchTST_supervised/plot_result")

result_df = pd.DataFrame(columns=["dataset", "country", "seq_length", "model", "name"])
for original_name in file_names:
    name = original_name.split(".")[0]
    dataset_country = name.split("_")[0]
    dataset = name.split("_")[1]
    model = name.split("_")[2]
    seq_len = int(name.split("_")[3])
    
    result_df = pd.concat([result_df, pd.DataFrame([[dataset, dataset_country, seq_len, model, f"/home/xuyang1/PatchTST/PatchTST_supervised/plot_result/{original_name}"]], columns=result_df.columns)], axis=0)



result_df = result_df.set_index(["dataset", "country", "model", "seq_length", ]).sort_index()
# result_df = result_df.set_index(["model"])
for dataset in result_df.index.get_level_values("dataset").drop_duplicates():
    part_df1 = result_df.loc[dataset]
    for country in part_df1.index.get_level_values("country").drop_duplicates():
        part_df2 = part_df1.loc[country]
        images = []
        for model in part_df2.index.get_level_values("model").drop_duplicates():
            part_df3 = part_df2.loc[model]
            for seq_length in part_df3.index.get_level_values("seq_length").drop_duplicates():
                part_df4 = part_df3.loc(axis=0)[seq_length]
                img = cv2.imread(part_df4["name"])
                cv2.putText(img, f"{dataset}_{country}_{seq_length}_{model}", (0, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                images.append(img)
        height, width, _ = images[0].shape  
        row_num = len(images) // 4
        col_num = 4
    
        # 创建一个空白图片，用于存放拼接后的图片  
        result_image = np.zeros((height * row_num, width * col_num, 3), np.uint8)  
    
        # 遍历每个图片，按照九宫格的顺序将其粘贴到结果图片上  
        for i, image in enumerate(images):  
            x = i % col_num * width  
            y = i // col_num * height  
            result_image[y:y+height, x:x+width] = image
        result_image = cv2.resize(result_image, (result_image.shape[1], result_image.shape[0]))
        cv2.imwrite(f'/home/xuyang1/PatchTST/PatchTST_supervised/plot_result_combined/{dataset}_{country}.png', result_image,)



display(result_df)


## 对比 self supervised

In [ ]:
import pandas as pd
import os
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import pickle
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

label_stocka_quintile = pd.read_csv(r'/home/xuyang1/PatchTST/PatchTST_supervised/dataset/quintile_stocka.csv', index_col=0)
label_us_quintile = pd.read_csv(r'/home/xuyang1/PatchTST/PatchTST_supervised/dataset/quintile_us.csv', index_col=0)
# label_stocka_quintile = pd.DataFrame(pickle.load(open(r'/home/xuyang1/PatchTST/PatchTST_supervised/dataset/quintile_stocka_transformed.pkl', 'rb')), index=label_stocka_quintile.index, columns=label_stocka_quintile.columns)
# label_us_quintile = pd.DataFrame(pickle.load(open(r'/home/xuyang1/PatchTST/PatchTST_supervised/dataset/quintile_us_transformed.pkl', 'rb')), index=label_us_quintile.index, columns=label_us_quintile.columns)
label_stocka_top10p = pd.read_csv(r'/home/xuyang1/PatchTST/PatchTST_supervised/dataset/top10p_stocka.csv', index_col=0)
label_us_top10p = pd.read_csv(r'/home/xuyang1/PatchTST/PatchTST_supervised/dataset/top10p_us.csv', index_col=0)
# label_stocka_top10p = pd.DataFrame(pickle.load(open(r'/home/xuyang1/PatchTST/PatchTST_supervised/dataset/top10p_stocka_transformed.pkl', 'rb')), index=label_stocka_top10p.index, columns=label_stocka_top10p.columns)
# label_us_top10p = pd.DataFrame(pickle.load(open(r'/home/xuyang1/PatchTST/PatchTST_supervised/dataset/top10p_us_transformed.pkl', 'rb')), index=label_us_top10p.index, columns=label_us_top10p.columns)

quintile_stocka_scaler = pickle.load(open(r'/home/xuyang1/PatchTST/PatchTST_supervised/dataset/quintile_stocka_scaler.pkl', 'rb'))
top10p_stocka_scaler = pickle.load(open(r'/home/xuyang1/PatchTST/PatchTST_supervised/dataset/top10p_stocka_scaler.pkl', 'rb'))
quintile_us_scaler = pickle.load(open(r'/home/xuyang1/PatchTST/PatchTST_supervised/dataset/quintile_us_scaler.pkl', 'rb'))
top10p_us_scaler = pickle.load(open(r'/home/xuyang1/PatchTST/PatchTST_supervised/dataset/top10p_us_scaler.pkl', 'rb'))
losses = []
for name in os.listdir(r"/home/xuyang1/PatchTST/PatchTST_self_supervised/results"):
    if "finetune" in name or "linear-probe" in name:
        model = "PatchTST" + name.split("_")[3]
        seq_length = name.split("tw")[-1].split("_")[0]
        dataset = name.split("_")[0]
        dataset_country = name.split("_")[1]
    else:
        model = name.split("_")[4]
        seq_length = name.split("_")[3]
        dataset = name.split("_")[0]
        dataset_country = name.split("_")[1]
    if dataset == "quintile" and dataset_country == "stocka":
        label = label_stocka_quintile
    elif dataset == "quintile" and dataset_country == "us":
        label = label_us_quintile
    elif dataset == "top10p" and dataset_country == "stocka":
        label = label_stocka_top10p
    elif dataset == "top10p" and dataset_country == "us":
        label = label_us_top10p
    pred = np.load(rf"/home/xuyang1/PatchTST/PatchTST_self_supervised/results/{name}/pred.npy")
    if "stocka" == dataset_country:
        pred = pred[:593, :, :]
    else:
        pred = pred[:986, :, :]

    if dataset == "quintile" and dataset_country == "stocka":
        scaler = quintile_stocka_scaler
    elif dataset == "quintile" and dataset_country == "us":
        scaler = quintile_us_scaler
    elif dataset == "top10p" and dataset_country == "stocka":
        scaler = top10p_stocka_scaler
    elif dataset == "top10p" and dataset_country == "us":
        scaler = top10p_us_scaler
    pred = scaler.inverse_transform(np.reshape(pred, (-1, pred.shape[-1]))).reshape(pred.shape)

    # if "quinti" not in dataset or seq_length != "96":
    #     continue

    test_length = int(len(label) * 0.2)

    for i in range(pred.shape[-1]):
        if label.values[:-test_length,i].mean() < 0:
            label.iloc[:,i] = label.iloc[:,i] * -1
            pred[:,:,i] = -pred[:,:,i]
    target_label = label.iloc[-test_length:,:]
    if pred.shape[0] > target_label.shape[0]:
        pred = pred[:target_label.shape[0],:,:]
    elif pred.shape[0] < target_label.shape[0]:
        target_label = target_label.iloc[:pred.shape[0],:]


    # calculate pred to real label loss:
    # loss = 0
    # for i in range(pred.shape[0]):
    #     pred_batch = pred[i]
    #     label_batch = target_label.iloc[i:i+pred.shape[1],:].values
    #     loss += np.mean((pred_batch - label_batch)**2)
    # mse = np.mean((pred[:,0,:] - target_label.values)**2)
    # mae = np.mean(np.abs(pred[:,0,:] - target_label.values))
    # print(f"{model},{dataset},{dataset_country},{seq_length},{mse},{mae}")
    # continue

    # calculate real return with prediction:
    # 1. use closest pred as real pread
    # real_pred = pred[:,0,:]

    # 2. use mean of pred as real pred
    # pred_middata_np = np.zeros((pred.shape[0], pred.shape[0] + pred.shape[1] - 1, pred.shape[2]))
    # pred_middata_np[:,:,:] = np.nan
    # for i in range(pred.shape[0]):
    #     pred_middata_np[i, i:i+pred.shape[1], :] = pred[i]
    # real_pred = np.nanmean(pred_middata_np, axis=0, keepdims=False)

    # 3. use mean of pred as real pred, but only use the first 20 pred
    real_pred = pred[:, :20, :].mean(axis=1, keepdims=False)

    pred_df = pd.DataFrame(real_pred, index=target_label.index[:real_pred.shape[0]], columns=target_label.columns)

    # ---- calculate rolling 20 days return ----
    rolling_20_label = target_label.rolling(20).mean().shift(-19).iloc(axis=0)[:-19]
    average_ret = rolling_20_label.mean(axis=1).to_frame("average_ret")

    past_rolling_20_label = target_label.rolling(20, closed="left", min_periods=1).mean()

    follow_past_20days_50p = rolling_20_label[(past_rolling_20_label.rank(axis=1, pct=True) > 0.5)].mean(axis=1).to_frame("follow_past_20days_50p")
    follow_past_20days_20p = rolling_20_label[(past_rolling_20_label.rank(axis=1, pct=True) > 0.8)].mean(axis=1).to_frame("follow_past_20days_20p")
    follow_past_20days_10p = rolling_20_label[(past_rolling_20_label.rank(axis=1, pct=True) > 0.9)].mean(axis=1).to_frame("follow_past_20days_10p")

    rolling_20_pred_df = pred_df.rolling(20).mean().shift(-20).iloc(axis=0)[:-20]
    top50p_pred_ret = rolling_20_label[(pred_df.rank(axis=1, pct=True) > 0.5)].mean(axis=1).to_frame("top50p_pred_ret")
    top20p_pred_ret = rolling_20_label[(pred_df.rank(axis=1, pct=True) > 0.8)].mean(axis=1).to_frame("top20p_pred_ret")
    top10p_pred_ret = rolling_20_label[(pred_df.rank(axis=1, pct=True) > 0.9)].mean(axis=1).to_frame("top10p_pred_ret")
    show_df = pd.concat([average_ret, top50p_pred_ret, top20p_pred_ret, top10p_pred_ret, follow_past_20days_50p, follow_past_20days_20p, follow_past_20days_10p], axis=1).cumsum(axis=0)
    show_df.plot()
    plt.title(f"{dataset_country} {dataset} {model} {seq_length}")
    # plt.show()

    # ---- plot pred and label in same photo ----
    # pred_with_input = pd.concat([label.iloc[:-test_length,:], pred_df], axis=0)
    # pred_with_input.columns = [col + "_pred" for col in pred_with_input.columns]
    # fig, axes = plt.subplots(5, 5)
    # fig.set_figheight(30)
    # fig.set_figwidth(30)
    # plt.subplots_adjust(wspace=None, hspace=None)
    # axes = axes.reshape(5, 5)
    # stride = label.shape[-1] // 25
    # for i in range(25):
    #     pred_and_gt = pd.concat([label.iloc(axis=1)[i*stride:i*stride+1], pred_with_input.iloc(axis=1)[i*stride:i*stride+1]], axis=1).cumsum(axis=0)
    #     pred_and_gt.plot(ax=axes[i//5, i%5], legend=True)
    #     axes[i//5, i%5].set_title(label.columns[i*stride])
    # plt.title(f"{dataset_country} {dataset} {model} {seq_length}")
    plt.savefig(rf"/home/xuyang1/PatchTST/PatchTST_self_supervised/plot_result/{dataset_country}_{dataset}_{model}_{seq_length}.png")
    # plt.show()
    
    # rics = []
    # for col in pred_df.columns:
    #     rics.append(pred_df[col].corr(target_label[col], method="spearman"))

    # print(dataset_country, dataset, seq_length, model, sum(rics) / len(rics), sep=",")
    plt.close()